In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/21 16:01:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/21 16:01:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/21 16:01:51 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
facets = spark.read.parquet("/Users/polina/Pathwaganda/data/search_facet_target")

In [11]:
facets.show(10)

+--------------------+-------------+-----------------+------------+
|               label|     category|        entityIds|datasourceId|
+--------------------+-------------+-----------------+------------+
|2'-5'-oligoadenyl...|Approved Name|[ENSG00000111335]|        NULL|
|3-hydroxybutyrate...|Approved Name|[ENSG00000161267]|        NULL|
|5'-aminolevulinat...|Approved Name|[ENSG00000158578]|        NULL|
|A-kinase anchorin...|Approved Name|[ENSG00000170776]|        NULL|
|A-kinase interact...|Approved Name|[ENSG00000166452]|        NULL|
| ABI family member 3|Approved Name|[ENSG00000108798]|        NULL|
|ACBD3 antisense R...|Approved Name|[ENSG00000234478]|        NULL|
|  ACCSL pseudogene 1|Approved Name|[ENSG00000230570]|        NULL|
|ACD shelterin com...|Approved Name|[ENSG00000102977]|        NULL|
|  ACTB pseudogene 12|Approved Name|[ENSG00000233125]|        NULL|
+--------------------+-------------+-----------------+------------+
only showing top 10 rows


In [12]:
facets.select("category").distinct().show()

+--------------------+
|            category|
+--------------------+
|       Approved Name|
|     Approved Symbol|
|           Target ID|
|               GO:CC|
|               GO:MF|
|               GO:BP|
|Subcellular Location|
| Tractability PROTAC|
|Tractability Othe...|
|Tractability Anti...|
|            Reactome|
|Tractability Smal...|
| ChEMBL Target Class|
+--------------------+



In [13]:
facets.filter(col("category") == "Reactome").show()

+--------------------+--------+--------------------+-------------+
|               label|category|           entityIds| datasourceId|
+--------------------+--------+--------------------+-------------+
|2-LTR circle form...|Reactome|[ENSG00000137309,...| R-HSA-164843|
|3-Methylcrotonyl-...|Reactome|[ENSG00000078070,...|R-HSA-9909438|
|3-hydroxyisobutyr...|Reactome|   [ENSG00000198130]|R-HSA-9916722|
|3-methylglutaconi...|Reactome|   [ENSG00000148090]|R-HSA-9914274|
|5-Phosphoribose 1...|Reactome|[ENSG00000229937,...|  R-HSA-73843|
|A tetrasaccharide...|Reactome|[ENSG00000038427,...|R-HSA-1971475|
|ABC transporters ...|Reactome|[ENSG00000162735,...|R-HSA-1369062|
|ABC-family protei...|Reactome|[ENSG00000187486,...| R-HSA-382556|
|ABO blood group b...|Reactome|[ENSG00000176920,...|R-HSA-9033807|
|ADORA2B mediated ...|Reactome|[ENSG00000065135,...|R-HSA-9660821|
|ADP signalling th...|Reactome|[ENSG00000114450,...| R-HSA-418592|
|ADP signalling th...|Reactome|[ENSG00000065135,...| R-HSA-392